In [1]:
# Start writing code here...

import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from igraph import *
import disjoint_set

This notebook contains template with example of data usage and loading (from prepared csv files, which were produced from original .xlsx file with the help of pandas)

In [2]:
if not ".." in sys.path:
    sys.path.insert(0, "..")

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
os.listdir("../data")

['sources.csv.gz',
 'input_wagon.xlsx',
 'orders7.csv.gz',
 'orders8.csv.gz',
 'wagon_mode_compat.csv.gz',
 'description.txt',
 'output_wagon.xlsx',
 'metrics.csv.gz',
 'standplaces.csv.gz']

In [5]:
orders7 = pd.read_csv("../data/orders7.csv.gz")
orders8 = pd.read_csv("../data/orders8.csv.gz")
wagon_mode_compat = pd.read_csv("../data/wagon_mode_compat.csv.gz")
sources = pd.read_csv("../data/sources.csv.gz")
standplaces = pd.read_csv("../data/standplaces.csv.gz")
metrics = pd.read_csv("../data/metrics.csv.gz")

In [6]:
# load data from file (this gets checked and reloaded because of autoreload magic above)
from utils.common import *



In [7]:
total_sum = 0
order_manager = OrderManager()
for index, row in orders7.iterrows():
    order = Order(row['Start'], row['Finish'], int(row['MaxUnit']), row['NeedWagonModel'], int(row['Tariff'].split(":")[0]), row['Dur'])
    order_manager.add_order(order)
    total_sum += order.total_profit()
print(total_sum)

wagon_manager = WagonManager()
for index, row in sources.iterrows():
    for units in range(row['Units']):
        occupied = True
        if int(row['Date']) == 0:
            occupied = False
        wagon = Wagon(row['WagonModel'], row['Station'], occupied, row['Date'])
        wagon_manager.add_wagon(wagon)

print(wagon_manager.wagon_number())


comp_wagon = set()
for index, row in wagon_mode_compat.iterrows():
    comp_wagon.add(str(row['NeedWagonModel']) + "_" + str(row['CompatibleWagonModel']))




434839750
15438


In [8]:

from igraph import *
def get_roads_graph(metrics):
    station_names = np.unique(
        np.concatenate([
            metrics.From.values,
            metrics.To.values
        ]))
    name2vertex = {str(x): i for i, x in enumerate(station_names)}
    graph = Graph()
    graph.add_vertices(len(station_names))
    columns = ["From", "To", "Group", "Distance", "Time", "PriceUnit"]
    for s, e, gr, d, t, price in metrics[columns].values:
        s = str(s)
        e = str(e)
        u = name2vertex[s]
        v = name2vertex[e]
        k = graph.add_edge(u, v, group=gr, distance=d, time=t, price_unit=price)
    return graph


graph = get_roads_graph(metrics)

KeyboardInterrupt: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7ba67d69-ce72-437b-8bf4-6d3c9b906c4a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>